# Exercice 3 : Méthodologie de validation et test

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.metrics import mean_squared_error

Choix de la fonction à apprendre pour l'exemple : ici un polynome de degré 5.

In [ ]:
listeParamPoly = [0.03, 0.2, -1, -10, 100]

In [ ]:
def generate_data(N):
    x = np.random.uniform(-10,10,N)
    y = np.polyval(listeParamPoly,x) + np.random.normal(0.0, 15.0, N)
    return x.reshape(-1, 1), y

Fonction permettant de calculer les intervalles de confiance de notre métrique (ici la [MSE](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html)):

In [ ]:
def confidence_interval(y, y_pred):
    n = len(y)
    s = np.sqrt(np.var(mean_squared_error(np.expand_dims(y,1).transpose(), np.expand_dims(y_pred,1).transpose(),
                          multioutput='raw_values'), ddof=1))
    return 1.96*s/np.sqrt(n)

Génération des données : Choisir le nombre de points à utiliser :

In [ ]:
dataPoints = 200

In [ ]:
X,y = generate_data(dataPoints)

In [ ]:
fig, ax = plt.subplots()
ax.plot(X, y, 'o')
ax.plot(np.linspace(-10,10,100), np.polyval(listeParamPoly,np.linspace(-10,10,100)), color='black', linewidth=3)
ax.set_title('Data set')
ax.set_ylabel('y')
ax.set_xlabel('x')
plt.show()

# Définition des fonctions de caractéristiques

Reprenez ici vos fonctions de caractéristiques de l'exercice précédent.

In [ ]:
phi = [
       lambda x: x**0, 
       lambda x: x**1,
       lambda x: x**2,
       lambda x: x**3,
       lambda x: np.abs(x),
       lambda x: x > -5.0,
       lambda x: x > 0.0,
       lambda x: x > 5.0,
       lambda x: np.cos(0.1*x),
       lambda x: np.cos(x),
       lambda x: np.sin(0.1*x),
       lambda x: np.sin(x),
      ]

In [ ]:
def feature_space_projection(X, phi):
    X_features = []
    for i in range(0, len(phi)):
        X_features.append(np.apply_along_axis(phi[i], 0, X))
    X_augmented = np.concatenate(X_features, axis=1)
    return X_augmented

# Préparation des données test

Après projection dans l'espace des caractéristiques, nous découpons notre dataset en trois parties.

In [ ]:
X = feature_space_projection(X, phi)

### splits : train(50%) - validation(25%) - test(25%)

Un ensemble d'entrainement, un ensemble de validation qui va nous permettre d'évaluer nos modèles et de choisir notre préféré sur un ensemble de données jamais vu, et un ensemble de test pour l'évaluation final en mode *réel*.

Remarquez que *théoriquement*, l'ensemble de valiation et l'ensemble de test sont identiques (sous l'hypothèse [IID](https://en.wikipedia.org/wiki/Independent_and_identically_distributed_random_variables)) et peuvent être permutés sans problème pour la méthodologie.

In [ ]:
X_, X_test, y_, y_test = train_test_split(X, y, test_size=0.25)
X_train, X_validation, y_train, y_validation = train_test_split(X_, y_, test_size=0.33)

### Entrainement du modèle sur les données d'entraînement

Vous pouvez ici changez les hyper-paramètres pour en voir l'effet sur les différentes erreurs sur les différents sous-ensembles de données. Vous pouvez aussi essayer d'autres modèles linéaires.

In [ ]:
reg2 = linear_model.ElasticNet(alpha=0.001, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=100000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='random', tol=0.0001, warm_start=False)
reg2.fit(X_train, y_train)

### Évaluation du modèle sur l'ensemble d'entraînement (training loss, erreur d'entrainement)

In [ ]:
y_train_pred = reg2.predict(X_train)
training_error = mean_squared_error(y_train, y_train_pred)
print("L'erreur d'entraînement du modèle appris est : %5.2f" % training_error)

### Évaluation du modèle sur l'ensemble de validation (validation loss, erreur de validation)

In [ ]:
y_val_pred = reg2.predict(X_validation)
validation_error = mean_squared_error(y_validation, y_val_pred)
print("L'erreur de validation du modèle appris est : %5.2f" % validation_error)

### Évaluation du modèle sur l'ensemble de test (test loss, erreur de test)

In [ ]:
y_test_pred = reg2.predict(X_test)
test_error = mean_squared_error(y_test, y_test_pred)
print("L'erreur de test du modèle appris est : %5.2f" % test_error)

### Estimation du vrai risque du modèle

In [ ]:
X_risk, y_risk = generate_data(1000000)
X_risk = feature_space_projection(X_risk, phi)
y_risk_pred = reg2.predict(X_risk)
true_risk = mean_squared_error(y_risk, y_risk_pred)
print("L'erreur de généralisation du modèle appris est : %5.2f ± %2.2f" % (true_risk, confidence_interval(y_risk, y_risk_pred)))

Si on se rappelle que l'ensemble de test et de validation sont statistiquement identiques, on concoit aisément que l'estimation du vrai risque est toujours très difficile et potentiellement éloignée de l'erreur de validation ou de celle de test. 

On a *optimisé* plusieurs modèles sur l'ensemble d'entrainement, qu'on en a *choisi* les hyperparamètres d'apprentissage optimaux sur l'ensemble de validation et qu'on a vérifié nos choix **une seule fois** sur l'ensemble de test. 